In [ ]:
# Requirments
!pip install langchain
!pip install transformers
!pip install streamlit
!pip install faiss-gpu
!pip install HuggingFace
!pip install faiss-cpu
!pip install sentence-transformers
!pip install ctransformers

import os
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import CTransformers
from langchain.chains import ConversationalRetrievalChain
#

In [ ]:
def main():
    DB_FAISS_PATH = "vectorstore/db_faiss"
    TEMP_DIR = "temp"

    if not os.path.exists(TEMP_DIR):
        os.makedirs(TEMP_DIR)

    csv_file_path = "bmw.csv"  # Update with CSV path required

    loader = CSVLoader(file_path=csv_file_path, encoding="utf-8", csv_args={'delimiter': ','}) # Just need to change delimiter if needed
    data = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(data)

    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    docsearch = FAISS.from_documents(text_chunks, embeddings)
    docsearch.save_local(DB_FAISS_PATH)

    llm = CTransformers(model="TheBloke/Llama-2-7B-Chat-GGML",
                        model_type="llama",
                        max_new_tokens=512,
                        temperature=0.1)

    qa = ConversationalRetrievalChain.from_llm(llm, retriever=docsearch.as_retriever())

    while True:
        query = input("Enter your query (type 'exit' to quit): ")
        if query.lower() == 'exit':
            break
        chat_history = []
        result = qa({"question": query, "chat_history": chat_history})
        print("Response:", result['answer'])

if __name__ == "__main__":
    main()